## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-16-23-48-28 +0000,bbc,Former Trump adviser John Bolton criminally in...,https://www.bbc.com/news/articles/cgql2qzkz5zo...
1,2025-10-16-23-44-07 +0000,nypost,Dr. Oz accuses Dems of ‘gaslighting’ Americans...,https://nypost.com/2025/10/16/media/dr-oz-accu...
2,2025-10-16-23-42-12 +0000,nyt,U.S. Military Killed Venezuelan Fisherman in S...,https://www.nytimes.com/2025/10/16/world/ameri...
3,2025-10-16-23-40-00 +0000,wsj,Teen Sues Maker of Fake-Nude Software,https://www.wsj.com/tech/ai/teen-sues-maker-of...
4,2025-10-16-23-35-20 +0000,nyt,John Bolton Indicted Over Handling of Classifi...,https://www.nytimes.com/2025/10/16/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,71
169,new,25
93,he,20
58,will,19
351,china,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
92,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,153
162,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...,144
24,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,143
362,2025-10-16-02-09-00 +0000,wsj,Democrats and Republicans both feel they have ...,https://www.wsj.com/politics/policy/government...,137
223,2025-10-16-14-49-58 +0000,nypost,Trump says CBS under owner David Ellison will ...,https://nypost.com/2025/10/16/media/trump-says...,136


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
92,153,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
362,66,2025-10-16-02-09-00 +0000,wsj,Democrats and Republicans both feel they have ...,https://www.wsj.com/politics/policy/government...
331,61,2025-10-16-09-00-00 +0000,wsj,What Happens When Oil and Gas Wells Die? West ...,https://www.wsj.com/business/energy-oil/what-h...
127,59,2025-10-16-19-13-22 +0000,bbc,At least three dead after Kenya police open fi...,https://www.bbc.com/news/articles/cwyk1l65nqvo...
269,45,2025-10-16-12-11-11 +0000,wapo,Hamas took 251 hostages from Israel into Gaza....,https://www.washingtonpost.com/world/interacti...
108,45,2025-10-16-20-21-13 +0000,nypost,Thune says ‘time’s right’ for Russia sanctions...,https://nypost.com/2025/10/16/us-news/thune-sa...
34,43,2025-10-16-22-31-31 +0000,startribune,"Prison for woman who admitted she was driving,...",https://www.startribune.com/prison-for-woman-a...
25,38,2025-10-16-22-53-17 +0000,nypost,Wannabe NYC killer sentenced after judge shoot...,https://nypost.com/2025/10/16/us-news/wannabe-...
369,36,2025-10-16-01-36-00 +0000,wsj,More than three dozen organizations and indivi...,https://www.wsj.com/politics/policy/trump-ball...
162,34,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
